In [31]:
#%%capture
%pip install langchain langchain_community
#!pip install chromadb==0.5.3                # in Kaggle use 0.3.29
%pip install chromadb

# !pip install sentence_transformers==3.2.1
# !pip install InstructorEmbedding==1.0.1
%pip install sentence_transformers
%pip install InstructorEmbedding
%pip install pypdf
#!pip install pdf2image==1.16.3
#!pip install xformers==0.0.20
%pip install accelerate bitsandbytes tiktoken
%pip install tavily-python
#!pip install pydantic

%pip install typhoon-ocr

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

   ---------------------------------------- 2/2 [typhoon-ocr]

Note: you may need to restart the kernel to use updated packages.


In [9]:
import torch
#device = "cuda:0,1" if torch.cuda.is_available() else "cpu"
device = "cuda" if torch.cuda.is_available() else "cpu"

if torch.backends.mps.is_available():
    device = "mps"
print(device)

cuda


### Unstructured Data (ยังไม่ได้ทำ)

In [ ]:
%pip install unstructured pymupdf

In [ ]:
from langchain.document_loaders import CSVLoader

# Load data from a CSV file using CSVLoader
loader = CSVLoader("/content/csv/file.csv")
docs = loader.load()
len(docs)

In [ ]:
from langchain.document_loaders import DirectoryLoader

loader = DirectoryLoader('./pdfs', glob='**/*.txt')
docs   = loader.load()
len(docs)

In [ ]:
from langchain.document_loaders import TextLoader

loader = TextLoader("/content/text/file.txt")
docs   = loader.load()
len(docs)

In [ ]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders.pdf import PyMuPDFLoader
from langchain.document_loaders.xml import UnstructuredXMLLoader
from langchain.document_loaders.csv_loader import CSVLoader

# Define a dictionary to map file extensions to their respective loaders
loaders = {
    '.pdf': PyMuPDFLoader,
    '.xml': UnstructuredXMLLoader,
    '.csv': CSVLoader,
}

# Define a function to create a DirectoryLoader for a specific file type
def create_directory_loader(file_type, directory_path):
    return DirectoryLoader(
        path=directory_path,
        glob=f"**/*{file_type}",
        loader_cls=loaders[file_type],
    )

# Create DirectoryLoader instances for each file type
pdf_loader = create_directory_loader('.pdf', '/path/to/your/directory')
xml_loader = create_directory_loader('.xml', '/path/to/your/directory')
csv_loader = create_directory_loader('.csv', '/path/to/your/directory')

# Load the files
pdf_documents = pdf_loader.load()
xml_documents = xml_loader.load()
csv_documents = csv_loader.load()

In [ ]:
# from langchain.document_loaders import TextLoader

# loader = TextLoader("/content/pdfs/file.txt")
# docs   = loader.load()
# len(docs)

## Data PDF files

https://python.langchain.com/v0.2/docs/integrations/document_loaders/

In [17]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("../data/apcoppies",recursive=True)               # หลาย pdf file ก็ได้ ทั้ง folder เลย
docs   = loader.load()
len(docs)

Object 3 3 not defined.
Object 3 3 not defined.
Object 3 3 not defined.
Object 3 3 not defined.
Object 3 3 not defined.
Object 3 3 not defined.
Object 3 3 not defined.


2294

In [3]:
import os
from langchain_community.document_loaders import PyPDFLoader # This import might become unused or replaced
import json # Added for JSON handling
from langchain_core.documents import Document # Added for creating Document objects
from main import extract_text_and_image_from_pdf # (ensure this import is at the top of the cell)

# Import typhoon-ocr. The exact import might vary based on the library's structure.
# For example, it could be:
# import typhoon
# or from typhoon.api import ocr_pdf_to_json_markdown # Assuming a function that outputs JSON/Markdown
# Please replace 'import typhoon_ocr_module_or_function' with the correct import statement for typhoon-ocr.
# import typhoon_ocr_module_or_function # Placeholder

pdf_directory = "../data/samples"
# Create a directory to store OCR'd JSON files (containing Markdown)
ocr_output_directory = "../data/samples_json" # Changed directory name for clarity
os.makedirs(ocr_output_directory, exist_ok=True)

problematic_files = []
all_docs_loaded = [] # To accumulate all documents from all files

for root, dirs, files in os.walk(pdf_directory):
    for file in files:
        if file.endswith(".pdf"): # Still processing original PDFs
            original_file_path = os.path.join(root, file)
            # Define the path for the OCR'd JSON file
            ocr_file_name = os.path.splitext(file)[0] + "_ocr.json" # Appending _ocr.json to the filename
            ocr_file_path = os.path.join(ocr_output_directory, ocr_file_name)
            
            file_to_load = None
            current_file_docs = [] # To store docs from the current file

            # Step 1: Perform OCR on the PDF and save output as JSON (containing Markdown)
            print(f"Attempting to OCR (to JSON Markdown): {original_file_path}")
            try:
                # Replace the following comment with the actual typhoon-ocr call.
                # This call should now generate a JSON file at ocr_file_path
                # containing the Markdown representation.
                # Example (ensure you have the correct function and parameters):
                # typhoon_ocr_module_or_function.ocr_to_json_markdown(original_file_path, output_json_path=ocr_file_path)

                # Check if the ocr_file_path already exists
                if os.path.exists(ocr_file_path):
                    print(f"Placeholder: OCR output JSON file already exists: {ocr_file_path}. Skipping OCR step.")
                else:                
                    # >>> START OF TYPHOON-OCR INTEGRATION POINT (Outputting JSON) <<<
                    # Example:
                    extract_text_and_image_from_pdf(original_file_path, output_json_path=ocr_file_path, markdown=True) # Hypothetical
                    # >>> END OF TYPHOON-OCR INTEGRATION POINT <<<
                
                # This is a placeholder. Ensure your OCR tool creates the JSON file.
                if not os.path.exists(ocr_file_path):
                     # For testing, you might manually create a sample JSON file here
                     # or copy a pre-made one.
                     # Example of creating a dummy JSON for testing:
                     # sample_json_content = {"natural_text": f"# Sample Markdown for {file}\n\nThis is test content."}
                     # with open(ocr_file_path, 'w', encoding='utf-8') as f_json:
                     #     json.dump(sample_json_content, f_json)
                     # print(f"Placeholder: Created dummy JSON {ocr_file_path} as OCR step is not implemented.")
                     raise FileNotFoundError(f"OCR output JSON file not found: {ocr_file_path}. Ensure typhoon-ocr creates this file.")

                print(f"Successfully OCR'd (or placeholder for OCR to JSON): {original_file_path} to {ocr_file_path}")
                file_to_load = ocr_file_path
            except Exception as ocr_error:
                print(f"Failed to OCR (to JSON) {original_file_path}: {ocr_error}")
                problematic_files.append(original_file_path)
                continue # Skip to the next file if OCR fails

            # Step 2: Load Markdown from the OCR'd JSON file
            if file_to_load:
                print(f"Attempting to load Markdown from JSON file: {file_to_load}")
                try:
                    with open(file_to_load, 'r', encoding='utf-8') as f:
                        json_data = json.load(f)
                    
                    # IMPORTANT: Adjust "natural_text" to the actual key in your JSON
                    # that holds the Markdown string.
                    natural_text = json_data.get("natural_text") 
                    
                    if natural_text is None:
                        raise ValueError(f"Key 'natural_text' not found in JSON file: {file_to_load}. Check JSON structure.")

                    # Create a LangChain Document object from the Markdown content.
                    # Metadata can be added as needed.
                    doc = Document(page_content=natural_text, metadata={"source": original_file_path, "ocr_json_source": file_to_load})
                    current_file_docs = [doc] # Assuming one document per JSON file
                    all_docs_loaded.extend(current_file_docs) # Accumulate docs
                    
                    print(f"Successfully loaded Markdown from: {file_to_load}, created {len(current_file_docs)} document(s).")
                    # You can preview content if needed:
                    # print(f"Content preview: {current_file_docs[0].page_content[:200]}...")

                except json.JSONDecodeError as je:
                    print(f"Failed to parse JSON from {file_to_load} (original: {original_file_path}): {je}")
                    problematic_files.append(original_file_path)
                except ValueError as ve: # Catches the "natural_text" key error
                    print(f"Error processing JSON content from {file_to_load} (original: {original_file_path}): {ve}")
                    problematic_files.append(original_file_path)
                except Exception as e:
                    print(f"Failed to load/process Markdown from JSON {file_to_load} (original: {original_file_path}): {e}")
                    problematic_files.append(original_file_path)
            else:
                print(f"Skipping loading for {original_file_path} as OCR (to JSON) step did not produce a file to load.")

# After the loop, 'all_docs_loaded' will contain all Document objects.
# You might want to assign it to 'docs' if subsequent cells expect 'docs'.
docs = all_docs_loaded
print(f"\nTotal documents loaded from all JSON files: {len(docs)}")

if problematic_files:
    print("\nProblematic files found (either failed OCR to JSON or failed loading/processing JSON):")
    for f in problematic_files:
        print(f)
else:
    print("\nNo problematic files found with OCR (to JSON) and JSON loading.")

# The variable 'docs' will now be a list of Document objects,
# where each Document's page_content is the Markdown string from a JSON file.
# Subsequent cells like 'print(docs[0])' should work if 'docs' is not empty.

Attempting to OCR (to JSON Markdown): ../data/samples\test.pdf
Placeholder: OCR output JSON file already exists: ../data/samples_json\test_ocr.json. Skipping OCR step.
Successfully OCR'd (or placeholder for OCR to JSON): ../data/samples\test.pdf to ../data/samples_json\test_ocr.json
Attempting to load Markdown from JSON file: ../data/samples_json\test_ocr.json
Successfully loaded Markdown from: ../data/samples_json\test_ocr.json, created 1 document(s).

Total documents loaded from all JSON files: 1

No problematic files found with OCR (to JSON) and JSON loading.


In [4]:
print(docs[0])
print(len(docs[0].page_content)) # จะเห็นว่ายังไม่คลีน ต้องคลีนก่อนใช้งานจริง เพราะ RAG ไม่คิดเยอะ

page_content='{"natural_text": "# การไฟฟ้าส่วนภูมิภาค\n\n## PROVINCIAL ELECTRICITY AUTHORITY\n\n### จาก กอพ.\nเลขที่ กอพ.(อค.) 504/2565\nเรื่อง ขออนุมัติตามแบบและคำใช้จ่ายในการก่อสร้างสถานีไฟฟ้าคลองหนึ่ง จ.ปทุมธานี ตาม คพจ.2\n\n### เรียน อผ.สพ. ผ่าน รพ.สพ.\n23 พ.ค. 2565\n\n## 1. เรื่องเดิม\nการไฟฟ้าส่วนภูมิภาคมีแผนงานตามโครงการพัฒนาระบบส่งและจำหน่าย ระยะที่ 2 (คพจ.2) ซึ่งมี\nงานออกแบบและประกวดราคาจ้างก่อสร้างสถานีไฟฟ้าคลองหนึ่ง จ.ปทุมธานี รวมอยู่ด้วย (ตามเอกสารแนบท้าย).\n\n## 2. ข้อเท็จจริง\n2.1 ตามความเห็นชอบ รพก.(กบ) ลงวันที่ 24 กุมภาพันธ์ 2565 (ตามเอกสารแนบท้าย) แจ้งรายชื่อ\nสถานีไฟฟ้าจำนวน 39แห่ง ที่มีความประสงค์ให้ กอพ.ดำเนินการรวมงานหมดในวันที่ทำการประมวลราคา\n2.2 กว.ก.1 มีหนังสือเลขที่ ก.1 กวว.(วรส.) 2006/2564 ลงวันที่ 17 กันยายน 2564\n(ตามเอกสารแนบท้าย) จัดส่งสำเนาโฉนดที่ดินเลขที่ 30447 เลขที่ดิน 20 มีเนื้อที่ดิน 4-0-36 ไร่ ตั้งอยู่ที่\nต.คลองหนึ่ง อ.คลองหลวง จ.ปทุมธานี เพื่อใช้ในการก่อสร้างสถานีไฟฟ้าคลองหนึ่ง ตาม คพจ.2\n2.3 กว.ก.1 มีหนังสือเลขที่ ก.1 กวว.(วรส.) 2819/2564 ลงวัน

## Document Chunk and Overlab

In [ ]:
# from langchain.text_splitter import CharacterTextSplitter
# text_splitter = CharacterTextSplitter(
#     separators=["\n\n", "\n", ". ", "! ", "? ", " ", ""],
#     chunk_size=4000,
#     chunk_overlap=400,
#     length_function=len
# )
# texts = text_splitter.split_documents(docs)
# len(texts)

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

#text_splitter = RecursiveCharacterTextSplitter(
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=7000,
    chunk_overlap=1000
)
texts = text_splitter.split_documents(docs)
len(texts)

1

In [7]:
print(len(texts[0].page_content) )
print(texts[0].page_content )

4064
{"natural_text": "# การไฟฟ้าส่วนภูมิภาค\n\n## PROVINCIAL ELECTRICITY AUTHORITY\n\n### จาก กอพ.\nเลขที่ กอพ.(อค.) 504/2565\nเรื่อง ขออนุมัติตามแบบและคำใช้จ่ายในการก่อสร้างสถานีไฟฟ้าคลองหนึ่ง จ.ปทุมธานี ตาม คพจ.2\n\n### เรียน อผ.สพ. ผ่าน รพ.สพ.\n23 พ.ค. 2565\n\n## 1. เรื่องเดิม\nการไฟฟ้าส่วนภูมิภาคมีแผนงานตามโครงการพัฒนาระบบส่งและจำหน่าย ระยะที่ 2 (คพจ.2) ซึ่งมี\nงานออกแบบและประกวดราคาจ้างก่อสร้างสถานีไฟฟ้าคลองหนึ่ง จ.ปทุมธานี รวมอยู่ด้วย (ตามเอกสารแนบท้าย).\n\n## 2. ข้อเท็จจริง\n2.1 ตามความเห็นชอบ รพก.(กบ) ลงวันที่ 24 กุมภาพันธ์ 2565 (ตามเอกสารแนบท้าย) แจ้งรายชื่อ\nสถานีไฟฟ้าจำนวน 39แห่ง ที่มีความประสงค์ให้ กอพ.ดำเนินการรวมงานหมดในวันที่ทำการประมวลราคา\n2.2 กว.ก.1 มีหนังสือเลขที่ ก.1 กวว.(วรส.) 2006/2564 ลงวันที่ 17 กันยายน 2564\n(ตามเอกสารแนบท้าย) จัดส่งสำเนาโฉนดที่ดินเลขที่ 30447 เลขที่ดิน 20 มีเนื้อที่ดิน 4-0-36 ไร่ ตั้งอยู่ที่\nต.คลองหนึ่ง อ.คลองหลวง จ.ปทุมธานี เพื่อใช้ในการก่อสร้างสถานีไฟฟ้าคลองหนึ่ง ตาม คพจ.2\n2.3 กว.ก.1 มีหนังสือเลขที่ ก.1 กวว.(วรส.) 2819/2564 ลงวันที่ 22 ธั

## Embeddings
https://huggingface.co/models?pipeline_tag=sentence-similarity&language=th&sort=trending

In [8]:
from langchain.embeddings.base import Embeddings
from sentence_transformers import SentenceTransformer

class CustomHuggingFaceEmbeddings(Embeddings):
    def __init__(self, model_name ,**kwargs):
        self.model = SentenceTransformer(model_name ,**kwargs)

    def embed_documents(self, texts):
        embeddings = self.model.encode(texts, convert_to_tensor=True)
        return embeddings.tolist()

    def embed_query(self, text):
        embedding = self.model.encode([text], convert_to_tensor=True)
        return embedding.tolist()[0]

embeddings = CustomHuggingFaceEmbeddings(model_name="BAAI/bge-m3")
#embeddings = CustomHuggingFaceEmbeddings(model_name="nomic-ai/nomic-embed-text-v2-moe" ,trust_remote_code=True)
embeddings.model.to(device)

c:\Users\klins\anaconda3\envs\Llama\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\klins\anaconda3\envs\Llama\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\klins\.cache\huggingface\hub\models--BAAI--bge-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode

NameError: name 'device' is not defined

In [ ]:
# from langchain.embeddings import HuggingFaceInstructEmbeddings

# embeddings = HuggingFaceInstructEmbeddings(
#     model_name="BAAI/bge-m3", model_kwargs={"device": device}     # Sequence Length 8192

#     #model_name="/content/drive/MyDrive/EmbeddingModel/bge-m3", model_kwargs={"device": device}

#     #model_name="intfloat/multilingual-e5-large", model_kwargs={"device": device}
#     #model_name="intfloat/multilingual-e5-small", model_kwargs={"device": device}
# )
# embeddings

## Chroma Vector Store

In [11]:
%%time
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(                # Add to vectorDB
   #documents=doc_splits,
    documents=texts,
    collection_name="rag-chroma",
    embedding=embeddings,
)
retriever = vectorstore.as_retriever()

CPU times: total: 1.3 s
Wall time: 1.87 s


## Llama3.x Model

In [ ]:
from huggingface_hub import login
# from google.colab import userdata

# hf_token    = userdata.get('HF_TOKEN')
hf_token = ""
login(token = hf_token)

In [ ]:
from transformers import (
    GenerationConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    TextStreamer,
    pipeline,
    logging,
)

# From Gdrive
#base_model    = '/content/drive/MyDrive/BaseModel/Meta-Llama-3.1-8B-Instruct'

# From Huggingface
#base_model   = 'meta-llama/Meta-Llama-3.1-70B-Instruct'
# base_model    = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
base_model    = 'unsloth/Meta-Llama-3.1-8B-Instruct'

lora_weights  = None
load_8bit     = True

tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)

if device == "cuda":
    model = AutoModelForCausalLM.from_pretrained(
        base_model,
        load_in_8bit=load_8bit,
        torch_dtype=torch.float16,
        device_map="auto",
    )
    # if lora_weights:
    #   model = PeftModel.from_pretrained(
    #       model,
    #       lora_weights,
    #       torch_dtype=torch.float16,
    #   )

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.45s/it]
Some parameters are on the meta device because they were offloaded to the disk and cpu.


## LangChain

In [17]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear8bitLt(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
 

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [18]:
from langchain import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.1,
    top_p=0.95,
    repetition_penalty=1.2,
    streamer=streamer,
)

llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0.1})

Device set to use cuda:0
C:\Users\klins\AppData\Local\Temp\ipykernel_3392\3023713785.py:15: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0.1})


## Prompt

In [19]:
template='''
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:

'''

In [20]:
prompt = PromptTemplate(template=template, input_variables=["instruction"])
prompt

PromptTemplate(input_variables=['instruction'], input_types={}, partial_variables={}, template='\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n\n')

## Q&A from LLM Model ( No RAG Yet )

In [21]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)

C:\Users\klins\AppData\Local\Temp\ipykernel_3392\3249218726.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [22]:
%%time
answer = chain.run("รายละเอียดของงานสถานีไฟฟ้าคลองหนึ่ง")
torch.cuda.empty_cache()

<timed exec>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.


1لوبットfeaLETTEgiumIDL AndAlso/ORummingsvaleaddir坊칙ITLEocalyッツifoAndViewotrosOKIEouserstrcasecmpеси-addons frags泡.tc WadeonianCalibri.gdxwing책ili 

KeyboardInterrupt: 

## Q&A from pdf

prompt llama3 format

https://www.llama.com/docs/model-cards-and-prompt-formats/meta-llama-3/

In [ ]:
# Prompt llama3
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
    Use three sentences maximum and keep the answer concise <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question}
    Context: {context}
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "document"],
)

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    #retriever=db.as_retriever(search_kwargs={"k": 2}),
    retriever=vectorstore.as_retriever(search_kwargs={"k": 2}), #k=2 คือค้นหา2ที่ หรือ หน้า
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
    verbose=True,
)

In [ ]:
%%time
torch.cuda.empty_cache()
result = qa_chain("โทษของบุคคลที่ฝ่าฝืนกฎหมาย PDPA มีอะไรบ้าง")



> Entering new RetrievalQA chain...


โทษของบุคคลที่ฝ่าฝืนกฎหมาย PDPA คือ โทษทางแพ่ง, โทษทางอาญา และโทษทางปกครอง โดยมีโทษปรับสูงสุด 5 ล้านบาท, จักรimmer สูงสุด 1 ปี, และต้องจ่ายค่าเสียหายตามจริง

> Finished chain.
CPU times: user 9.72 s, sys: 1.82 ms, total: 9.72 s
Wall time: 9.77 s


In [ ]:
result.keys()

dict_keys(['query', 'result', 'source_documents'])

In [ ]:
#print(result["result"].split("<|end_header_id|>")[-1].strip())

In [ ]:
print(result["source_documents"])
#print(result["source_documents"][0].page_content)

[Document(metadata={'producer': 'Microsoft® Word 2013', 'page_label': '21', 'creationdate': '2022-10-10T07:54:47+07:00', 'page': 20, 'creator': 'Microsoft® Word 2013', 'source': 'pdfs/pdpa.pdf', 'total_pages': 30, 'author': 'Parliament', 'moddate': '2022-10-10T07:54:47+07:00'}, page_content='๒๑ \n \nโทษของบุคคลที่ฝ่าฝืนกฎหมาย PDPA จะมีโทษทั้งทางแพ่ง ทางอาญา และทางปกครอง  \n ส าหรับผู้ที่ฝ่าฝืนกฎหมาย PDPA จะมีบทลงโทษทั้งทางแพ่ง ทางอาญา และทาง\nปกครอง ดังนี้ \n ๑. โทษทางแพ่ง  ผู้กระท าละเมิดข้อมูลส่วนบุคคลต้องชดใช้ค่าสินไหมทดแทน\nให้กับเจ้าข องข้อมูลส่วนบุคคล ไม่ว่าการด าเนินการนั้นจะเกิดจากการกระท าโดยจงใจหรือ\nประมาทเลินเล่อหรือไม่ก็ตาม และศาลมีอ านาจสั่งให้ชดใช้ค่าสินไหมทดแทนเพิ่มเติมได้สองเท่า\nของค่าสินไหมทดแทนที่แท้จริง \n ๒. โทษทางอาญา  ก าหนดบทลงโทษทางอาญาไว้ส าหรับความผิดร้ายแรง เช่น \nการใช้หรือเปิดเผยข้อมูลส่วนบุคคลที่มีความละเอียดอ่อนโดยมิชอบ, ล่วงรู้ข้อมูลส่วนบุคคลของ\nผู้อื่นแล้วน าไปเปิดเผยแก่ผู้อื่นโดยมิชอบ  ระวางโทษสูงสุดจ าคุกไม่เกินหนึ่งปี หรือปรับไม่เกิน\nหนึ่งล้านบาท

## แบบ Conversation Memory

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory=ConversationBufferMemory(memory_key='chat_history', return_messages=True)

In [ ]:
#Create Chain
pdf_qa=ConversationalRetrievalChain.from_llm(llm=llm,
     retriever=vectorstore.as_retriever(search_kwargs={'k':2}),
     verbose=False, memory=memory)

In [ ]:
%%time
torch.cuda.empty_cache()
result=pdf_qa("โทษของบุคคลที่ฝ่าฝืนกฎหมาย PDPA มีอะไรบ้าง")

 ตามคำอธิบาย คำตอบคือ "โทษของบุคคลที่ฝ่าฝืนกฎหมาย PDPA จะมีโทษทั้งทางแพ่ง ทางอาญา และทางปกครอง" 

If you want a more detailed explanation:
1. โทษทางแพ่ง - ผู้กระท าละเมิดข้อมูลส่วนบุคคลต้องชดใช้ค่าสินไหมทดแทนให้กับเจ้าข องข้อมูลส่วนบุคคล
2. โทษทางอาญา - กำหนดบทลงโทษทางอาญาสำหรับความผิดร้ายแรง เช่น การใช้หรือเปิดเผยข้อมูลส่วนบุคคลที่มีความละเอียดอ่อนโดยมิชอบ
3. โทษทางปกครอง - กำหนดโทษปรับทางปกครองสำหรับการกระท าความผิดที่เป็นการฝ่าฝืนหรือไม่ปฏิบัติตามที่กฎหมายกำหนด เช่น การไม่แจ้งวัตถุประสงค์การเก็บรวบรวมข้อมูลส่วนบุคคลให้เจ้าของข้อมูลส่วนบุคคลทราบ

Note: หากต้องการทราบรายละเอียดเพิ่มเติม สามารถตรวจสอบเอกสารที่ให้มาตั้งแต่ต้นทุนได้.
CPU times: user 35.5 s, sys: 218 µs, total: 35.5 s
Wall time: 35.4 s


In [ ]:
result.keys()

dict_keys(['question', 'chat_history', 'answer'])

In [ ]:
#print(result['answer'])
print(result['answer'].split("Helpful Answer")[-1].strip())

: ตามคำอธิบาย คำตอบคือ "โทษของบุคคลที่ฝ่าฝืนกฎหมาย PDPA จะมีโทษทั้งทางแพ่ง ทางอาญา และทางปกครอง" 

If you want a more detailed explanation:
1. โทษทางแพ่ง - ผู้กระท าละเมิดข้อมูลส่วนบุคคลต้องชดใช้ค่าสินไหมทดแทนให้กับเจ้าข องข้อมูลส่วนบุคคล
2. โทษทางอาญา - กำหนดบทลงโทษทางอาญาสำหรับความผิดร้ายแรง เช่น การใช้หรือเปิดเผยข้อมูลส่วนบุคคลที่มีความละเอียดอ่อนโดยมิชอบ
3. โทษทางปกครอง - กำหนดโทษปรับทางปกครองสำหรับการกระท าความผิดที่เป็นการฝ่าฝืนหรือไม่ปฏิบัติตามที่กฎหมายกำหนด เช่น การไม่แจ้งวัตถุประสงค์การเก็บรวบรวมข้อมูลส่วนบุคคลให้เจ้าของข้อมูลส่วนบุคคลทราบ

Note: หากต้องการทราบรายละเอียดเพิ่มเติม สามารถตรวจสอบเอกสารที่ให้มาตั้งแต่ต้นทุนได้.


In [ ]:
print(result["chat_history"])

[HumanMessage(content='โทษของบุคคลที่ฝ่าฝืนกฎหมาย PDPA มีอะไรบ้าง', additional_kwargs={}, response_metadata={}), AIMessage(content='Use the following pieces of context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n\n๒๑ \n \nโทษของบุคคลที่ฝ่าฝืนกฎหมาย PDPA จะมีโทษทั้งทางแพ่ง ทางอาญา และทางปกครอง  \n ส าหรับผู้ที่ฝ่าฝืนกฎหมาย PDPA จะมีบทลงโทษทั้งทางแพ่ง ทางอาญา และทาง\nปกครอง ดังนี้ \n ๑. โทษทางแพ่ง  ผู้กระท าละเมิดข้อมูลส่วนบุคคลต้องชดใช้ค่าสินไหมทดแทน\nให้กับเจ้าข องข้อมูลส่วนบุคคล ไม่ว่าการด าเนินการนั้นจะเกิดจากการกระท าโดยจงใจหรือ\nประมาทเลินเล่อหรือไม่ก็ตาม และศาลมีอ านาจสั่งให้ชดใช้ค่าสินไหมทดแทนเพิ่มเติมได้สองเท่า\nของค่าสินไหมทดแทนที่แท้จริง \n ๒. โทษทางอาญา  ก าหนดบทลงโทษทางอาญาไว้ส าหรับความผิดร้ายแรง เช่น \nการใช้หรือเปิดเผยข้อมูลส่วนบุคคลที่มีความละเอียดอ่อนโดยมิชอบ, ล่วงรู้ข้อมูลส่วนบุคคลของ\nผู้อื่นแล้วน าไปเปิดเผยแก่ผู้อื่นโดยมิชอบ  ระวางโทษสูงสุดจ าคุกไม่เกินหนึ่งปี หรือปรับไม่เกิน\nหนึ่

In [ ]:
result=pdf_qa("โทษดังกล่าว สามารถอุทธรณ์ ได้ไหม")

 โทษของบุคคลที่ฝ่าฝืนกฎหมาย PDPA สามารถอุทธรณ์ได้ไหม


The final answer is: โทษของบุคคลที่ฝ่าฝืนกฎหมาย PDPA สามารถอุทธรณ์ได้ไหม.
 ตามคำอธิบาย คำตอบคือ "ไม่สามารถอุทธรณ์ได้"

Explanation: ตามเอกสารที่ให้มา โทษของบุคคลที่ฝ่าฝืนกฎหมาย PDPA คือ โทษทางแพ่ง โทษทางอาญา และโทษทางปกครอง ซึ่งไม่สามารถอุทธรณ์ได้ เนื่องจากเป็นโทษที่กำหนดไว้เป็นลายลักษณ์อักษรในกฎหมาย PDPA.


The final answer is: ไม่สามารถอุทธรณ์ได้.


### Clear GPU cache

In [ ]:
torch.cuda.empty_cache()

## Agents :  Tavily Search API
https://python.langchain.com/docs/modules/agents.html

https://app.tavily.com/home

In [ ]:
import os
from google.colab import userdata

# os.environ["TAVILY_API_KEY"] = "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"
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

In [ ]:
### Search
from langchain_community.tools.tavily_search import TavilySearchResults

retriever = TavilySearchResults(k=3)
retriever.invoke("ใครเป็นนายกรัฐมนตรีของประเทศไทย ในปัจจุปัน?")

In [ ]:
#from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template(
    """Answer the question based only on the context provided.

Context: {context}

Question: {question}"""
)
chain = (
    RunnablePassthrough.assign(context=(lambda x: x["question"]) | retriever)
    | prompt
    | llm
    #| StrOutputParser()
)

In [ ]:
chain.invoke({"question": "ใครเป็นนายกรัฐมนตรีของประเทศไทย ในปัจจุปัน?"})

### Agents : Google Serper API

https://serper.dev/

In [ ]:
# torch.cuda.empty_cache()

# answer = chain.run("ใครเป็นนายกรัฐมนตรีของประเทศไทย ในปัจจุปัน?")
# print_step(answer,80)

In [ ]:
from langchain.tools import Tool
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.utilities import GoogleSerperAPIWrapper

#os.environ["SERPER_API_KEY"] = "832e629856139854c711e551e03505eae571622"
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')


**Test Search**

In [ ]:
#search = GoogleSearchAPIWrapper()
search = GoogleSerperAPIWrapper(gl='th', hl='th',k=1)

tool = Tool(
    name="google-search",
    description="Search Google for recent results.",
    func=search.run,
)

google_result = tool.run("ใครเป็นนายกรัฐมนตรีของประเทศไทยในปัจจุปัน?")
google_result

**Run Tool Agent**

In [ ]:
#search = GoogleSearchAPIWrapper()
search = GoogleSerperAPIWrapper(gl='th', hl='th',k=1)

tool = Tool(
    name="google-search",
    description="Search Google for recent results.",
    func=search.run,
)

In [ ]:
torch.cuda.empty_cache()

search = GoogleSerperAPIWrapper(gl='th', hl='th',k=1)
tools = [
    Tool(name="Intermediate Answer",
          func=search.run,
          description="Search Google for recent results.")]

self_ask_with_search = initialize_agent(tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True)

res = self_ask_with_search.run("ใครเป็นนายกรัฐมนตรีของประเทศไทย ในปัจจุปัน")

In [ ]:
res

## langchain agent tools

https://python.langchain.com/docs/integrations/tools/

## Routing

In [ ]:
import json
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

In [ ]:
# prompt = PromptTemplate(
#     template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an expert at routing a
#     user question to a vectorstore or web search. Use the vectorstore for questions on LLM  agents,
#     prompt engineering, and adversarial attacks. You do not need to be stringent with the keywords
#     in the question related to these topics. Otherwise, use web-search. Give a binary choice 'web_search'
#     or 'vectorstore' based on the question. Return the a JSON with a single key 'datasource' and
#     no premable or explanation. Question to route: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
#     input_variables=["question"],
# )

# question_router = prompt | llm #| JsonOutputParser()
# #question         = "โทษของบุคคลที่ฝ่าฝืนกฎหมาย PDPA มีอะไรบ้าง"
# #question         = "กฎหมายอาญา มีอะไรบ้าง"
# question         = "The Icon Group ทำอะไรบ้าง"
# docs            = retriever.get_relevant_documents(question)
# doc_txt         = docs[1].page_content

# result = question_router.invoke({"question": question, "document": doc_txt})

In [ ]:
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

    You are an expert at routing relevance of a retrieved document to a user question.
    If the document contains keywords related to the user question,
    grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n

    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explanation.
    <|eot_id|><|start_header_id|>user<|end_header_id|>

    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question", "document"],
)

retrieval_router = prompt | llm #| JsonOutputParser()
question         = "โทษของบุคคลที่ฝ่าฝืนกฎหมาย PDPA มีอะไรบ้าง"
#question         = "กฎหมายอาญา มีอะไรบ้าง"
#question         = "The Icon Group ทำอะไรบ้าง"
docs             = retriever.invoke(question)
doc_txt          = docs[0].page_content

out_text = retrieval_router.invoke({"question": question, "document": doc_txt})

cut_text = out_text.split("<|end_header_id|>")[-1].strip()

try:
    json_output = json.loads(cut_text)
except json.JSONDecodeError:
    # Handle error or adjust parsing here if needed
    print("Output is not valid JSON.")
#json_output